In [1]:
import os
os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "3,4,5,6"  # 指定使用 GPU 3
import torch

In [3]:
import os
import gc
import re
import ast
import sys
import copy
import json
import time
import math
import string
import pickle
import random
import joblib
import itertools
import warnings
warnings.filterwarnings("ignore")
# import keras

import scipy as sp
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import StratifiedKFold, GroupKFold, KFold

import torch
import torch.nn as nn
from torch.nn import Parameter
import torch.nn.functional as F
from torch.optim import Adam, SGD, AdamW
from torch.utils.data import DataLoader, Dataset

import tokenizers
import transformers
print(f"tokenizers.__version__: {tokenizers.__version__}")
print(f"transformers.__version__: {transformers.__version__}")
from transformers import AutoTokenizer, AutoModel, AutoConfig
from transformers import get_linear_schedule_with_warmup, get_cosine_schedule_with_warmup
from transformers import DataCollatorWithPadding
from transformers import AutoModelForSequenceClassification, BertForSequenceClassification
%env TOKENIZERS_PARALLELISM=false

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

tokenizers.__version__: 0.19.1
transformers.__version__: 4.41.1
env: TOKENIZERS_PARALLELISM=false


In [4]:
import pandas as pd 
from transformers import (
    AutoTokenizer, 
    AutoModelForSequenceClassification, 
    Trainer, 
    TrainingArguments, 
    DataCollatorWithPadding
)
from datasets import Dataset
from glob import glob
import gc
import torch
from scipy.special import softmax

## 数据load和处理

In [5]:
import polars as pl
columns = [  
    (
        pl.col("full_text").str.split(by="\n\n").alias("paragraph")
    ),
]
PATH = "kaggle/input/learning-agency-lab-automated-essay-scoring-2/"
TEST_DATA_PATH = "kaggle/input/learning-agency-lab-automated-essay-scoring-2/test.csv"
MAX_LENGTH = 512
EVAL_BATCH_SIZE = 1


## 添加模型

In [6]:
# Load model directlyfrom transformers import AutoTokenizer, AutoModelForSequenceClassification
tokenizer = AutoTokenizer.from_pretrained("/home/mcq/GitHub/aes2/kaggle/input/argument-classifier")
model = AutoModelForSequenceClassification.from_pretrained("/home/mcq/GitHub/aes2/kaggle/input/argument-classifier")


In [7]:
PATH = '/home/mcq/GitHub/aes2/kaggle/input/learning-agency-lab-automated-essay-scoring-2/'

train = pl.read_csv(PATH + "train.csv").with_columns(columns) 
test = pl.read_csv(PATH + "test.csv").with_columns(columns)

In [8]:
from sklearn.model_selection import train_test_split  # Import package

train, valid = train_test_split(train, test_size=0.2, stratify= train['score'])

### 小型预处理


In [9]:
Y1 =pd.DataFrame({ "labels":train['score'] })
Y2 =pd.DataFrame({ "labels":valid['score'] })
# y_split = Y1.astype(int).values
# labels = y_split.tolist()

def cut(tmp):   
    tmp = tmp.with_columns(pl.col('full_text').str.slice(0, 513).alias
    ("cut"))
    return tmp

train = cut(train)
valid = cut(valid)

df_train =pd.DataFrame({ "cut": train['cut']})
df_valid = pd.DataFrame({ "cut": valid['cut']})

df_train = pd.concat([df_train, Y1], axis=1)
df_valid = pd.concat([df_valid, Y2], axis=1)

def tokenize(sample):
    return tokenizer(sample['cut'], max_length=512, padding='max_length', truncation=True)

ds_train = Dataset.from_pandas(df_train)
ds_train = ds_train.map(tokenize)
ds_train = ds_train.remove_columns(['cut'])


ds_valid = Dataset.from_pandas(df_valid)
ds_valid= ds_valid.map(tokenize)
ds_valid = ds_valid.remove_columns(['cut'])

Map:   0%|          | 0/13845 [00:00<?, ? examples/s]

Map:   0%|          | 0/3462 [00:00<?, ? examples/s]

In [10]:
import torch.nn as nn

class ModifiedRobertaClassificationHead(nn.Module):
    def __init__(self):
        super(ModifiedRobertaClassificationHead, self).__init__()
        self.dense = nn.Linear(768, 768)
        self.dropout = nn.Dropout(p=0.1)
        self.out_proj = nn.Linear(768, 1)  # Changed from 2 to 6

    def forward(self, features):
        x = self.dense(features)
        x = nn.functional.relu(x)
        x = self.dropout(x)
        x = self.out_proj(x)
        return x
    
model.classifier = ModifiedRobertaClassificationHead()

In [11]:
model.classifier

ModifiedRobertaClassificationHead(
  (dense): Linear(in_features=768, out_features=768, bias=True)
  (dropout): Dropout(p=0.1, inplace=False)
  (out_proj): Linear(in_features=768, out_features=1, bias=True)
)

### 训练过程

In [12]:
#preds = trainer.predict(ds).predictions

In [13]:
#predictions = []
#predictions.append(softmax(preds, axis=-1))

In [14]:
# from transformers import Trainer, TrainingArguments

# training_args = TrainingArguments(
#     output_dir='./results',          # output directory
#     num_train_epochs=5,              # number of training epochs
#     per_device_train_batch_size=8,   # batch size for training
#     per_device_eval_batch_size=1,   # batch size for evaluation
#     warmup_steps=500,                # number of warmup steps for learning rate scheduler
#     weight_decay=0.01,               # strength of weight decay
#     logging_dir='./logs',            # directory for storing logs
#     logging_steps=10,
# )

# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=dataset
# )

In [15]:

#     
#     )

# trainer = Trainer(
#         model=model, 
#         args=args, 
#         train_dataset=ds,
#         #data_collator=DataCollatorWithPadding(tokenizer), 
#         tokenizer=tokenizer
#     )
train_loader = DataLoader(ds_train, batch_size=32, shuffle=True)
args = TrainingArguments(
     ".", 
     per_device_train_batch_size=32,
     per_device_eval_batch_size=EVAL_BATCH_SIZE, 
     fp16=True,                       # 使用 16 位精度
     dataloader_num_workers=4,   
     report_to="none"
 )
# training_args = TrainingArguments(
#    output_dir='./results',          # output directory
#     num_train_epochs=5,  )            # number of training epochs
    #per_device_train_batch_size=32,   # batch size for training
    #per_device_eval_batch_size=1,   # batch size for evaluation
    #warmup_steps=500,                # number of warmup steps for learning rate scheduler
    #weight_decay=0.01,               # strength of weight decay
    #logging_dir='./logs',            # directory for storing logs
    #logging_steps=10,)


trainer = Trainer(
    model=model,
    args= args,
    train_dataset=ds_train,
    eval_dataset=ds_valid,
    tokenizer= tokenizer
    )

In [16]:
trainer.train()

ValueError: Caught ValueError in replica 0 on device 0.
Original Traceback (most recent call last):
  File "/home/mcq/anaconda3/envs/torch/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py", line 83, in _worker
    output = module(*input, **kwargs)
  File "/home/mcq/anaconda3/envs/torch/lib/python3.10/site-packages/torch/nn/modules/module.py", line 1532, in _wrapped_call_impl
    return self._call_impl(*args, **kwargs)
  File "/home/mcq/anaconda3/envs/torch/lib/python3.10/site-packages/torch/nn/modules/module.py", line 1541, in _call_impl
    return forward_call(*args, **kwargs)
  File "/home/mcq/anaconda3/envs/torch/lib/python3.10/site-packages/transformers/models/roberta/modeling_roberta.py", line 1229, in forward
    loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
  File "/home/mcq/anaconda3/envs/torch/lib/python3.10/site-packages/torch/nn/modules/module.py", line 1532, in _wrapped_call_impl
    return self._call_impl(*args, **kwargs)
  File "/home/

In [ ]:
torch.cuda.empty_cache()
gc.collect()

0

In [ ]:
predictions = []
for model in models:
    model = AutoModelForSequenceClassification.from_pretrained(model)
    trainer = Trainer(
        model=model, 
        args=args, 
        data_collator=DataCollatorWithPadding(tokenizer), 
        tokenizer=tokenizer
    )
    
    preds = trainer.tokenizer.predict(ds).predictions
    predictions.append(softmax(preds, axis=-1))  #predictions.append(...) 将转换后的概率分布添加到 predictions 列表中，用于进一步处理或评估。
    del model, trainer
    torch.cuda.empty_cache()
    gc.collect()
    
# predicted_score = 0.

# for p in predictions:
#     predicted_score += p
    
# predicted_score /= len(predictions)

In [ ]:
# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_dataset['train'],
    eval_dataset=encoded_dataset['test']
)

# Train and evaluate
trainer.train()
trainer.evaluate()

# Save the fine-tuned model
model.save_pretrained("fine-tuned-model")


